<a href="https://colab.research.google.com/github/DhruvSaraogi/OOP-Project/blob/main/notebooks/Chest_xray_localization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Save a copy of this Notebook before execution.

Check GPU configuration.
In this project Nvidia tesla P100 GPU was used.

In [1]:
!nvidia-smi

Tue Dec 10 16:33:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Clone the Repository

In [2]:
!git clone https://github.com/gavin-d26/Localization-of-Lung-abnormalities-on-Chest-X-Rays-using-self-supervised-equivariant-attention.git

Cloning into 'Localization-of-Lung-abnormalities-on-Chest-X-Rays-using-self-supervised-equivariant-attention'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (37/37), done.
Receiving objects: 100% (53/53), 803.62 KiB | 396.00 KiB/s, done.
remote: Total 53 (delta 23), reused 36 (delta 14), pack-reused 0 (from 0)
Resolving deltas: 100% (23/23), done.


Create a kaggle API token and upload the downloaded kaggle.json to the content directory.

In [3]:
from google.colab import files
files.upload()

KeyboardInterrupt: 

Create a directory named kaggle that will store the kaggle.json file,
and display list of kaggle datasets.


In [ ]:
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

Create a directory to save models

In [ ]:
!mkdir saved_models

1.   Change current directory to project directory.
2.   Download nih-chest-xrays dataset as a data.zip file into project directory.
1.   Unzip the data.zip file.










In [ ]:
%cd Localization-of-Lung-abnormalities-on-Chest-X-Rays-using-self-supervised-equivariant-attention
!mkdir data
%cd data
!kaggle datasets download -d nih-chest-xrays/data
!unzip \*.zip  && rm *.zip
%cd ..

Install necessary packages

In [7]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


Login to Weights and Biases (wandb) for experiment tracking.

In [ ]:
!wandb login

Next, in weights and biases, create a new project titled "chest-xray-localization".

Run the trainpcm.py script with the following options.

1.   --savedirectory : Path to save directory
2.   --model : Type of model (resnet50, resnet50PCM, efficientnetb4, efficientnetb4PCM)
3.   --name : Name of current run
4.   --project : Name of wandb project
5.   --entity : Name of wandb entity (username) (for more info [link](https://docs.wandb.ai/ref/python/init))



In [ ]:
%cd /content/Localization-of-Lung-abnormalities-on-Chest-X-Rays-using-self-supervised-equivariant-attention
!python scripts/trainpcm.py --savedirectory /content/saved_models --model resnet50PCM --name resnet50PCM_v1 --project chest-xray-localization --entity YOUR_WANDB_USERNAME

During Training, two versions of the model are saved. Final (_FW_) and Checkpoint (_CW_). Checkpoint refers to the model with highest Validation AUROC Score.
Go to wandb to view results.


1.   For classification, see AUROC_MACRO_TEST_Checkpoint in summary section.
2.   For localization, see table titled RUNNAME-CW-default/pcm/pcm-rv_Checkpoint



In [5]:
!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchmetrics==0.9.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 GB 774.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.7/419.7 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1+cu121
    Uninstalling torch-2.5.1+cu121:
      Successfully uninstalled torch-2.5.1+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.20.1+cu121
    Uninstalling torchvision-0.20.1+cu121:
      Successfully uninstalled torchvision-0.20.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.13.2 requires torch>=1.13.0, but you have torch 1.11.0+cu113 which is incompatible.
torchaudio 2.5.1+cu121 requires torch==2.5.1, but you have torch

In [9]:
!pip install -r requirements.txt


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
